In [161]:
import random
from collections import Counter

class Bag:
    def __init__(self, tile_counts, out_of_bag):
        self.out_of_bag = out_of_bag

        self.tiles = []
        for i, count in enumerate(tile_counts):
            self.tiles.extend([i] * count)
        random.shuffle(self.tiles)

    def pop(self):
        if len(self.tiles) == 0:
            # Refill the bag
            self.__init__(self.out_of_bag, [0] * 5)
        return self.tiles.pop()

def draw_factories(bag, num_factories=5):
    """
    Draw tiles from the bag and fill the factories.
    Returns a list of the factories.
    """
    factories = [[] for _ in range(num_factories)]

    for factory in factories:
        # Draw 4 tiles for each factory
        for _ in range(4):
            factory.append(bag.pop())
    
    # Sort the factories to make it easier to compare
    return sorted([tuple(sorted(factory)) for factory in factories])

def simulate(num_runs, num_factories=5):
    """
    Simulate drawing tiles from the bag and filling the factories.
    Returns a Counter of the number of times each factory occurs.
    """
    outcomes = []

    for i in range(num_runs):
        bag = Bag([20, 20, 20, 20, 20], [0] * 5)
        outcomes.append(tuple(draw_factories(bag, num_factories)))

        if i % 4096 == 0:
            print(i, end="\r")
    
    # Count the frequency of each outcome
    outcome_counts = Counter(outcomes)
    
    # Find the most common outcomes
    most_common = outcome_counts.most_common(5)

    return most_common

simulate(100000)

# # Tile counts for each color
# tile_counts = [3, 0, 0, 0, 0]
# out_of_bag = [17, 20, 20, 20, 20]
# bag = Bag(tile_counts, out_of_bag)
# print(bag.tiles)
# draw_factories(bag)

[(((0, 1, 1, 3), (0, 1, 1, 4), (0, 1, 2, 2), (1, 2, 3, 4), (2, 3, 4, 4)), 4),
 (((0, 1, 2, 3), (0, 1, 2, 4), (0, 1, 3, 4), (1, 2, 3, 4), (1, 2, 3, 4)), 4),
 (((0, 1, 1, 3), (0, 1, 2, 3), (0, 1, 2, 4), (0, 1, 2, 4), (0, 3, 3, 4)), 4),
 (((0, 1, 2, 2), (0, 1, 3, 4), (0, 2, 3, 4), (1, 2, 3, 3), (1, 2, 3, 4)), 4),
 (((0, 1, 4, 4), (0, 2, 3, 3), (0, 2, 3, 4), (1, 3, 4, 4), (1, 3, 4, 4)), 3)]

In [3]:
from itertools import combinations_with_replacement
from collections import Counter
from math import comb

def multichoose(n, k):
    """ Calculate the number of ways to choose k items from n types, allowing repetition. """
    return comb(n + k - 1, k)

def calculate_probability(tile_counts, combination):
    """ Calculate the probability of drawing a specific combination of tiles. """
    total_tiles = sum(tile_counts)
    probability = 1
    for tile in combination:
        probability *= tile_counts[tile] / total_tiles
        total_tiles -= 1
        tile_counts[tile] -= 1
    return probability

def simulate_factories(num_factories, initial_tile_counts):
    """ Simulate the factories and calculate probabilities of outcomes. """
    all_combinations = list(combinations_with_replacement(range(5), 4))
    print(len(all_combinations))
    outcomes = Counter()

    comb = combinations_with_replacement(all_combinations, num_factories)
    #print(len(list(comb)))
    for i, factory_layout in enumerate(comb):
        if i % 10000 == 0:
            print(i, end="\r")
        tile_counts = initial_tile_counts.copy()
        probability = 1
        for factory in factory_layout:
            probability *= calculate_probability(tile_counts, factory)
        outcomes[factory_layout] += probability

    # Find the most common outcomes
    return outcomes.most_common(5)

# Initial counts of each tile type
initial_tile_counts = [20, 20, 20, 20, 20]

# Simulate the factories
most_common_outcomes = simulate_factories(5, initial_tile_counts)
for outcome in most_common_outcomes:
    print(outcome)

70
(((0, 0, 0, 0), (1, 1, 1, 1), (2, 2, 2, 3), (2, 3, 3, 3), (4, 4, 4, 4)), 1.6302311350513592e-14)
(((0, 0, 0, 0), (1, 1, 1, 1), (2, 2, 2, 4), (2, 4, 4, 4), (3, 3, 3, 3)), 1.6302311350513592e-14)
(((0, 0, 0, 0), (1, 1, 2, 2), (1, 1, 2, 2), (3, 3, 3, 3), (4, 4, 4, 4)), 1.6302311350513592e-14)
(((0, 0, 0, 0), (1, 1, 2, 3), (1, 2, 2, 2), (1, 3, 3, 3), (4, 4, 4, 4)), 1.6302311350513592e-14)
(((0, 0, 0, 0), (1, 1, 2, 3), (1, 2, 2, 3), (1, 2, 3, 3), (4, 4, 4, 4)), 1.6302311350513592e-14)


0

In [9]:
def get_game_result(score1, score2):
    NORMALIZATION_SCORE_LIMIT = 100
    BASE_SCORE_ADJUSTMENT = 0.001
    COMPLEMENTARY_SCORE_ADJUSTMENT = 1 - BASE_SCORE_ADJUSTMENT

    score_difference = float(score1) - float(score2)
    print(score_difference)
    normalized_score_difference = abs(score_difference) / NORMALIZATION_SCORE_LIMIT * BASE_SCORE_ADJUSTMENT
    print(normalized_score_difference)

    if score_difference > 0:
        return COMPLEMENTARY_SCORE_ADJUSTMENT + normalized_score_difference
    elif score_difference < 0:
        return BASE_SCORE_ADJUSTMENT - normalized_score_difference
    else:
        return 0.5

# Example usage
result = get_game_result(60, 3)
print(result)

57.0
0.00057
0.99957


In [28]:

def from_game_scores(game_scores, NUM_PLAYERS, BASE_SCORE_ADJUSTMENT, COMPLEMENTARY_SCORE_ADJUSTMENT):
    max_score = max(game_scores)
    min_score = min(game_scores)

    score_range = max_score - min_score
    if score_range == 0:
        # If all scores are the same, return 1 / NUM_PLAYERS for each player
        return [1.0 / NUM_PLAYERS for _ in range(NUM_PLAYERS)]

    value = [0.0] * NUM_PLAYERS
    for i, score in enumerate(game_scores):
        normalized_score = (score - min_score) / score_range
        value[i] = normalized_score

    sum_value = sum(value)
    return [v / sum_value for v in value]

NUM_PLAYERS = 4
game_scores = [2, 3, 2, 1] # Example scores for 4 players

values = from_game_scores(game_scores, NUM_PLAYERS, BASE_SCORE_ADJUSTMENT, COMPLEMENTARY_SCORE_ADJUSTMENT)
values

[0.25, 0.5, 0.25, 0.0]

In [3]:
class Value:
    def __init__(self, values):
        self.values = values

    @classmethod
    def from_game_scores(cls, game_scores, num_players):
        max_score = max(game_scores)
        min_score = min(game_scores)

        score_range = max_score - min_score
        if score_range == 0:
            # If all scores are the same, return equal value for each player
            return cls([1.0 / num_players] * num_players)

        # Normalize scores
        normalized_scores = [(score - min_score) / score_range for score in game_scores]

        # Normalize so the sum of all values is 1
        sum_scores = sum(normalized_scores)
        # normalized_scores = [score / sum_scores for score in normalized_scores]

        return cls(normalized_scores)

# Example usage:
num_players = 4
game_scores = [10, 30, 30, 40]
value_instance = Value.from_game_scores(game_scores, num_players)
print(value_instance.values)


[0.0, 0.6666666666666666, 0.6666666666666666, 1.0]
